# Pre-processing Colorado Allocation data for WaDEQA upload.
Date Updated: 04/13/2022
Purpose:  To pre-process the Colorado data into one master file for simple DataFrame creation and extraction

Notes:
asdf

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Colorado/WaterAllocation/RawInputData"
os.chdir(workingDir)

# Input File
fileInput = "DWR_Water_Right_-_Net_Amounts_input.csv"
df = pd.read_csv(fileInput)
df.head(2)

C:\Users\rjame\AppData\Local\Temp\ipykernel_21320\392003879.py:7: DtypeWarning: Columns (12,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fileInput)


,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,9/27/2002,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)"
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,7/26/1968,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)"


In [3]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
df['Appropriation Date'] = pd.to_datetime(df['Appropriation Date'])
df['Appropriation Date'] = pd.to_datetime(df["Appropriation Date"].dt.strftime('%m/%d/%Y'))
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)"
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)"


In [4]:
#Creating Beneficial Use.
#Need to split CO abbreviatoin strings to a workable format.

BenUseDict = {
"0" : "Storage",
"1" : "Irrigation",
"2" : "Municipal",
"3" : "Commercial",
"4" : "Industrial",
"5" : "Recreation",
"6" : "Fishery",
"7" : "Fire",
"8" : "Domestic",
"9" : "Stock",
"A" : "Augmentation",
"B" : "Export from Basin",
"C" : "Cumulative Accretion to River",
"D" : "Cumulative Depletion from River",
"E" : "Evaporative",
"F" : "Federal Reserved",
"G" : "Geothermal",
"H" : "Household Use Only",
"K" : "Snow Making",
"M" : "Minimum Streamflow",
"N" : "Net Effect on River",
"P" : "Power Generation",
"Q" : "Other",
"R" : "Recharge",
"S" : "Export from State",
"T" : "Transmountain Export",
"W" : "Wildlife",
"X" : "All Beneficial Uses"}

def retrieveBenUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = "Unspecified"
    else:
        String1 = str(colrowValue).strip()
        x=[]
        x[:] = String1
        try:
            outList = []
            for i in range(len(x)):
                y = x[i].strip()
                y = BenUseDict[y]
                outList.append(y)
            outString = ",".join(str(e) for e in outList)
        except:
            outString = "Unspecified"
    return outString

df['in_WaDEBenUse'] = df.apply(lambda row: retrieveBenUse(row['Decreed Uses']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio..."
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock"


In [5]:
#Determining WaterSourceTypeCV

def determineWaterSourceTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = "Unspecified"
    else:
        String1 = str(colrowValue).strip()
        if "GROUNDWATER:" in String1:
            outString = "Groundwater"
        else:
            outString = "Surface Water"
    return outString

df['in_WaterSourceTypeCV'] = df.apply(lambda row: determineWaterSourceTypeCV(row['Water Source']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water


In [6]:
#Determining WaterSourceName
#Want to remove the "GROUNDWATER: " from the name.

def determineWaterSourceName(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = "Unspecified"
    else:
        String1 = str(colrowValue).strip()
        if "GROUNDWATER:" in String1:
            outString = colrowValue.replace("GROUNDWATER: ", "").strip()
        else:
            outString = colrowValue
    return outString

df['in_WaterSourceName'] = df.apply(lambda row: determineWaterSourceName(row['Water Source']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK


In [7]:
#Filling in blank spots of GNIS ID for use of WaterSourceNativeID

def fillGNISID(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = "Unspecified"
    else:
        outString = str(colrowValue).strip()
    return outString

df['GNIS ID'] = df.apply(lambda row: fillGNISID(row['GNIS ID']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK


In [8]:
# Allocation_CFS
# If Decreed Units = "C" and Net Absolute != 0, then return Net Absolute
# Elif Decreed Units = "C" and Net Conditional != 0, then return Net Conditional
# Else return blank

# For creating Allocation_CFS
def assignAllocation_CFS(valA, valB, valC):
    valA = str(valA).strip()
    if (valB != 0) and (valC != 0):
        outString = 0
    else:
        if (valA == "C") and (valB != 0):
            outString = valB
        elif (valA  == "C") and (valC != 0):
            outString = valC
        else:
            outString = 0
    return outString

df['in_AllocationFlow_CFS'] = df.apply(lambda row: assignAllocation_CFS(row["Decreed Units"], row["Net Absolute"], row["Net Conditional"]), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName,in_AllocationFlow_CFS
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK,0.00
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK,0.27


In [9]:
# AllocationVolume_AF
# If Decreed Units = "A" and Net Absolute != 0, then return Net Absolute
# Elif Decreed Units = "A" and Net Conditional != 0, then return Net Conditional
# Else return blank

# For creating AllocationVolume_AF
def assignAllocationVolume_AF(valA, valB, valC):
    valA = str(valA).strip()
    if (valB != 0) and (valC != 0):
        outString = 0
    else:
        if (valA == "A") and (valB != 0):
            outString = valB
        elif (valA  == "A") and (valC != 0):
            outString = valC
        else:
            outString = 0
    return outString

df['in_AllocationVolume_AF'] = df.apply(lambda row: assignAllocationVolume_AF(row["Decreed Units"], row["Net Absolute"], row["Net Conditional"]), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName,in_AllocationFlow_CFS,in_AllocationVolume_AF
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK,0.00,0.0
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK,0.27,0.0


In [10]:
# For creating AllocationLegalStatusCV
# If Net Absolute = 0 and Net Condontial = 0, then Condtional Aboslute
# Elif Net Absolute = 0 and Net Condontial != 0, then Condtional
# Else, Aboslute

def assignAllocationLegalStatusCV(valA, valB):
    if (valA == 0) and (valB == 0):
        outString = "Conditional Absolute"
    elif (valA == 0) and (valB != 0):
        outString = "Conditional"
    else:
        outString = "Absolute"
    return outString

df['in_AllocationLegalStatusCV'] = df.apply(lambda row: assignAllocationLegalStatusCV(row['Net Absolute'], row['Net Conditional']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK,0.00,0.0,Absolute
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK,0.27,0.0,Absolute


In [11]:
# Need a unique identifier for WaDE AllocationNativeID.  Combine combine **Admin No**, **Order No**, **Decreed Units**, & **WDID** into single string entry.

# For creating AllocationAmount
def assignAllocationNativeID(colrowValueA, colrowValueB, colrowValueC, colrowValueD):
    outString = "-".join(map(str, [colrowValueA, colrowValueB, colrowValueC, colrowValueD]))
    return outString

df['in_AllocationNativeID'] = df.apply(lambda row: assignAllocationNativeID(row['Admin No'], row['Order No'], row['Decreed Units'], row['WDID']), axis=1)
df.head(2)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.00000,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK,0.00,0.0,Absolute,55787.0-0-C-6801433
1,6800885,JUTTEN SPRING & PIPELINE,Spring,ONION CREEK,188246.0,NaN,4,68,OURAY,NW,NE,NE,19,47.0 N,7.0 W,N,NaN,NaN,NaN,NaN,266380.8,4245051.4,38.323168,-107.672448,Spotted from quarters,12/31/1972,12/31/1971,1968-07-26,44559.43306,0,NaN,W0532,89,0.270,0.000,0.0,0.0,C,No,"ONION CREEK TRIB W532 IN PLACE USE, ONION CR W532",1/4/2005 10:00,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.323168, -107.672448)","Domestic,Stock",Surface Water,ONION CREEK,0.27,0.0,Absolute,44559.43306-0-C-6800885


In [12]:
df['in_WaterAllocationNativeURL'] = df['More Information']
df.head(1)

,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,in_WaDEBenUse,in_WaterSourceTypeCV,in_WaterSourceName,in_AllocationFlow_CFS,in_AllocationVolume_AF,in_AllocationLegalStatusCV,in_AllocationNativeID,in_WaterAllocationNativeURL
0,6801433,QUAKING ASPENS DITCH,Ditch,COAL CREEK,188598.0,0.81,4,68,OURAY,NaN,SE,NE,4,44.0 N,8.0 W,N,NaN,NaN,NaN,NaN,259910.0,4220032.0,38.096228,-107.737935,GPS,12/31/2002,12/31/2001,2002-09-27,55787.0,0,NaN,"16CW0049, 02CW0246",158PW,0.033,0.078,0.0,0.0,C,No,50 GPM CUMULATIVE WITH PATSY'S & BRITTNEY'S SP...,4/6/2017 16:53,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(38.096228, -107.737935)","Irrigation,Recreation,Domestic,Power Generatio...",Surface Water,COAL CREEK,0.0,0.0,Absolute,55787.0-0-C-6801433,https://dwr.state.co.us/Tools/WaterRights/NetA...


## Export Outputs

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)

WDID                                    int64
Structure Name                         object
Structure Type                         object
Water Source                           object
GNIS ID                                object
Stream Mile                           float64
DIV                                     int64
WD                                      int64
County                                 object
Q10                                    object
Q40                                    object
Q160                                   object
Section                                object
Township                               object
Range                                  object
PM                                     object
CoordsEW                              float64
CoordsEW Dir                           object
CoordsNS                              float64
CoordsNS Dir                           object
UTM x                                 float64
UTM y                             

In [14]:
#Exporting to Finished File
df.to_csv('P_ColoradoMaster.csv', index=False)  # The output